In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
df_org = pd.read_csv('docs/t20_bbb.csv')

In [ ]:
df_org=df_org[(df_org['ball']<=11)]
df_org=df_org[(df_org['over']<=20)]
df_org=df_org[(df_org['batruns']>=0) & (df_org['bowlruns']>=0)]
df_org.shape

In [ ]:
def bowl_style_clean(df,unique_styles):

    df['bowl_style'] = df['bowl_style'].replace(['RFM', 'RMF'], 'RFM')
    df['bowl_style'] = df['bowl_style'].replace(['LFM', 'LMF'], 'LFM')
    df['bowl_style'] = df['bowl_style'].replace(['LBG', 'LB'], 'RWS')
    df['bowl_style'] = df['bowl_style'].replace(['LS', 'SLA'], 'SLA')
    df['bowl_style'] = df['bowl_style'].replace(['RAB'], 'RSM')
    df['bowl_style'] = df['bowl_style'].replace(['LAB'], 'LSM')
    unique_styles = ['LFM/SLA', 'OB/LB','LM/SLA/LWS','OB/SLA','RMF/OB', 'RFM/OB', 'OB/LBG', 'RFM/LBG', 'RM/LB',
           'RM/LBG', 'LMF/RM', 'RM/RSM', 'SLA/LWS', 'RMF/LB', 'RFM/LB',
           'RM/OB/LB']
    replacements = {
        'RFM': 'RFM',
        'RMF': 'RFM',
        'LFM': 'LFM',
        'LMF': 'LFM',
        'LBG': 'RWS',
        'LB': 'RWS',
        'LS': 'SLA',
        'SLA': 'SLA',
        'RAB': 'RS',
        'LAB': 'LS'
    }

    # Iterate over unique styles and apply replacements
    for style in unique_styles:
        replaced_style = '/'.join([replacements.get(part, part) for part in style.split('/')])
        df['bowl_style'] = df['bowl_style'].replace([style], replaced_style)
    #df['bowl_style'].unique()
    return df
unique_styles = ['RFM', 'RMF', 'LBG', 'LB', 'LS', 'RAB', 'LAB', 'OB', 'SLA', 'RAB','RSM','LSM']
df=bowl_style_clean(df_org,unique_styles)
df['bowl_style'].unique()

In [ ]:
df_worldcup = pd.read_csv('docs/2024 wc squad.csv', encoding='latin1')
df_worldcup.head()

In [ ]:
team_dict = df_worldcup.to_dict(orient='list')

print(team_dict['Afghanistan'])

In [ ]:
team_dict = {key.strip(): [value.strip() for value in values if pd.notna(value)] for key, values in team_dict.items()}

team_dict['Afghanistan']

In [ ]:
players_list = np.concatenate((df['bowl'].unique(), df['bat'].unique()))
for team,players in team_dict.items():
    for player in players:
        if ((player not in players_list)|(player not in players_list)):
            print(team,player)

In [ ]:
players=set()
for key,vals in team_dict.items():
    players.update(vals)

In [ ]:
matches_relevant=df[df['bat'].isin(players) | df['bowl'].isin(players)]['p_match'].unique()
df=df[df['p_match'].isin(matches_relevant)]

In [ ]:
df_player_id_combs=df[['bat','p_bat','bowl','p_bowl']].drop_duplicates()

In [ ]:
# Create a mapping of player names to player IDs
player_id_map = {}
for index, row in df_player_id_combs.iterrows():
    if row['bat'] in player_id_map:
        continue
    player_id_map[row['bat']] = row['p_bat']
    player_id_map[row['bowl']] = row['p_bowl']
# Initialize lists to store data
player_names = []
countries = []
player_ids = []

# Iterate over the dictionary
for country, players in team_dict.items():
    for player in players:
        player_names.append(player)
        countries.append(country)
        player_ids.append(player_id_map.get(player, None))

# Create DataFrame from lists
team_df = pd.DataFrame({'Player': player_names, 'Country': countries, 'p_id': player_ids})
team_df['p_id'] = team_df['p_id'].astype('Int64')
team_df.head()

In [ ]:
worldcup_pitches=['Kensington Oval, Bridgetown, Barbados',
                  'Sir Vivian Richards Stadium, North Sound, Antigua',
                  'Providence Stadium, Guyana',
                  'Daren Sammy National Cricket Stadium, Gros Islet, St Lucia',
                  'Brian Lara Stadium, Tarouba, Trinidad',
                  'Arnos Vale Ground, Kingstown, St Vincent',
                  'Central Broward Regional Park Stadium Turf Ground, Lauderhill',
                  'Nassau County International Cricket Stadium',
                  'Grand Prairie Stadium'
                 ]

In [ ]:
df.columns

In [ ]:
def custom_agg(x):
    result = (x['batruns'].sum() * 120) / x['ballfaced'].sum()
    return round(result, 2)
def get_ground_avgscores(df):
    df1=df[df['ground'].isin(worldcup_pitches)].groupby(['p_match', 'inns','ground']).agg({'batruns':'sum','ballfaced':'sum'}).reset_index()
    # Define custom aggregation function
    # Group by 'inns' and 'ground', and aggregate using custom aggregation function
    result_df = df1.groupby(['ground']).apply(custom_agg).reset_index(name='average_score')

    return result_df
df_groundavg=get_ground_avgscores(df)
df_groundavg

In [ ]:
# Create a dictionary for ground indexes
ground_index = {ground: f'G{i}' for i, ground in enumerate(df_groundavg['ground'].unique(), 1)}

# Add the index to the dataframe
df_groundavg['ground_index'] = df_groundavg['ground'].map(ground_index)

plt.figure(figsize=(14, 8))

# Create the bar plot
ax = sns.barplot(x='ground_index', y='average_score', data=df_groundavg, palette='viridis')

# Annotate bars with ground names outside and scores inside
for p, label in zip(ax.patches, df_groundavg['ground']):
    height = p.get_height()
    ax.annotate(format(height, '.2f'), 
                (p.get_x() + p.get_width() / 2., height), 
                ha = 'center', va = 'center', 
                xytext = (8, -15), 
                textcoords = 'offset points', color='white')
    
    ax.annotate(label, 
                (p.get_x() + p.get_width() / 2., 5), 
                ha = 'center', va = 'bottom', 
                fontsize=9, rotation=90, color='white')

# Title and labels
plt.title('Average Scores by Ground')
plt.xlabel('Grounds')
plt.ylabel('Average Score')

# # Create the legend for ground names
# handles, labels = plt.gca().get_legend_handles_labels()
# plt.legend(handles, ['Innings 1', 'Innings 2'], title='Innings')

plt.show()

In [ ]:
df_groundavg.to_csv('docs/WC Analysis results/scoreavg_ground.csv')

In [ ]:
def get_ground_batstats_vs_bowlkind(df):
    df2=df[(df['ground'].isin(worldcup_pitches))&(df['bowl_kind'].isin(['pace bowler','spin bowler']))]
    df2_runs=df2.groupby(['ground','bowl_kind']).agg({'batruns':'sum','ballfaced':'sum','p_match':'nunique'}).reset_index()
    df2_outs=df[(df['out']==True)&(df['ground'].isin(worldcup_pitches))&(df['bowl_kind'].isin(['pace bowler','spin bowler']))].groupby(['ground','bowl_kind']).size().reset_index(name='outs')
    df2_batting=pd.merge(df2_runs,df2_outs,on=['ground','bowl_kind'])
    df2_batting['average']=round(df2_batting['batruns']/df2_batting['outs'],2)
    df2_batting['strike rate']=round(df2_batting['batruns']*100/df2_batting['ballfaced'],2)
    df2_batting.drop(columns=['batruns','ballfaced','outs'],inplace=True)
    df2_batting.rename(columns={'p_match':'matches'},inplace=True)
    return df2_batting
df2_batting=get_ground_batstats_vs_bowlkind(df)
df2_batting.head()

In [ ]:
ground_index = {ground: f'G{i}' for i, ground in enumerate(df2_batting['ground'].unique(), 1)}

# Add the index to the dataframe
df2_batting['ground_index'] = df2_batting['ground'].map(ground_index)

plt.figure(figsize=(14, 8))

# Create the bar plot
ax = sns.barplot(x='ground_index', y='strike rate', hue='bowl_kind', data=df2_batting, palette='viridis')

# Annotate bars with ground names outside and scores inside
for p, label in zip(ax.patches, df2_batting['ground']):
    height = p.get_height()
    ax.annotate(format(height, '.2f'), 
                (p.get_x() + p.get_width() / 2., height), 
                ha = 'center', va = 'center', 
                xytext = (0, -10), 
                textcoords = 'offset points', color='white')
    
    ax.annotate(label, 
                (p.get_x() + p.get_width() / 2., 5), 
                ha = 'center', va = 'bottom', 
                fontsize=9, rotation=90, color='white')

# Title and labels
plt.title('Average Strike Rate by Ground and Bowling Kinds')
plt.xlabel('Grounds')
plt.ylabel('Batting Strike rate')

# Create the legend for ground names
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles, ['Pace bowling', 'Spin Bowling'], title='Bowling Kind')

plt.show()

In [ ]:
df2_batting.to_csv("docs/WC Analysis results/batting_bowlkind_by_ground.csv")

In [ ]:
def get_battingstats(df):
    df3=pd.merge(df[df['bat'].isin(team_df[team_df['p_id'].isna()==False]['Player'])],team_df[team_df['p_id'].isna()==False],left_on='p_bat',right_on='p_id',how='inner')
    df3_bat=df3.groupby(['Country']).agg({'batruns':'sum','ballfaced':'sum','p_match':'nunique'}).reset_index()
    df3_outs=df3[df3['out']==True].groupby(['Country']).size().reset_index(name='outs')
    df3_bat_stats=pd.merge(df3_bat,df3_outs,on='Country')
    df3_bat_stats['bat_avg']=round(df3_bat_stats['batruns']/df3_bat_stats['outs'],2)
    df3_bat_stats['bat_sr']=round(df3_bat_stats['batruns']*100/df3_bat_stats['ballfaced'],2)
    df3_dots=df3[(df3['bowlruns']==0) & (df3['ballfaced']==1)].groupby(['Country']).agg({'ballfaced':'sum'}).reset_index()
    df3_dots.rename(columns={'ballfaced':'dotballs'},inplace=True)
    df3_bat_stats=pd.merge(df3_bat_stats,df3_dots,on="Country")
    df3_bat_stats['dot%']=(df3_bat_stats['dotballs']*100/df3_bat_stats['ballfaced']).round(2)
    def custom_agg(x):
        boundary_runs = x['batruns'].sum()
        return boundary_runs
    # Apply the custom aggregation function and reset index
    df3_bd = df3[((df3['batruns'] == 4) | (df3['batruns'] == 6)) & (df3['ballfaced'] == 1)] \
                .groupby(['Country']).apply(custom_agg).reset_index(name='boundary_runs')
    df3_bat_stats=pd.merge(df3_bat_stats,df3_bd,on="Country")
    df3_bat_stats['boundary%']=(df3_bat_stats['boundary_runs']*100/df3_bat_stats['batruns']).round(2)
    df3_bat_stats.drop(columns=['batruns', 'ballfaced', 'dotballs','boundary_runs','outs'],inplace=True)
    df3_bat_stats.rename(columns={'p_match':'matches_batted'},inplace=True)
    return df3_bat_stats
df3_bat_stats=get_battingstats(df)
df3_bat_stats.head()

In [ ]:
def get_bowling_stats(df):
    df4=pd.merge(df[df['bowl'].isin(team_df[team_df['p_id'].isna()==False]['Player'])],team_df[team_df['p_id'].isna()==False],left_on='p_bowl',right_on='p_id',how='inner')
    df4_bowl=df4.groupby(['Country']).agg({'bowlruns':'sum','ballfaced':'sum','p_match':'nunique'}).reset_index()
    df4_bowl['economy']=(df4_bowl['bowlruns']*6/df4_bowl['ballfaced']).round(2)
    df4_outs=df4[df4['out']==True].groupby(['Country']).size().reset_index(name='outs')
    df4_bowl_stats=pd.merge(df4_bowl,df4_outs,on='Country')
    df4_bowl_stats['bowl_avg']=round(df4_bowl_stats['bowlruns']/df4_bowl_stats['outs'],2)
    df4_bowl_stats['bowl_sr']=round(df4_bowl_stats['ballfaced']/df4_bowl_stats['outs'],2)
    df4_bowl_stats.drop(columns=['bowlruns','ballfaced','outs'],inplace=True)
    df4_bowl_stats.rename(columns={'p_match':'matches_bowled'},inplace=True)
    return df4_bowl_stats
df4_bowl_stats=get_bowling_stats(df)
df4_bowl_stats.head()

In [ ]:
overall_team_stats1=pd.merge(df3_bat_stats,df4_bowl_stats,on='Country').sort_values('matches_batted',ascending=False).reset_index().drop(columns='index').head(10)
overall_team_stats1.set_index('Country', inplace=True)

overall_team_stats2=pd.merge(df3_bat_stats,df4_bowl_stats,on='Country').sort_values('matches_batted',ascending=False).reset_index().drop(columns='index').tail(9)
overall_team_stats2.set_index('Country', inplace=True)

In [ ]:
overall_team_stats1

In [ ]:
overall_team_stats2

In [ ]:
overall_team_stats1.drop(columns=['matches_batted','matches_bowled'],inplace=True)
overall_team_stats2.drop(columns=['matches_batted','matches_bowled'],inplace=True)

In [ ]:
overall_team_stats=pd.concat([overall_team_stats1,overall_team_stats2],ignore_index=False)
overall_team_stats.to_csv("docs/WC Analysis results/overall_team_stats.csv")

In [ ]:
cols_increase=['bat_avg','bat_sr','boundary%']
cols_decrease=['dot%','economy','bowl_avg','bowl_sr']
def normalization(cols_increase, cols_decrease, overall_team_stats):
    # Create an empty DataFrame for normalized values
    normalized_df = pd.DataFrame(index=overall_team_stats.index)
    
    # Normalize columns where an increase is favorable
    for col in cols_increase:
        mean = overall_team_stats[col].mean()
        std = overall_team_stats[col].std()
        normalized_df[col] = (overall_team_stats[col] - mean) / std

    # Normalize columns where a decrease is favorable
    for col in cols_decrease:
        mean = overall_team_stats[col].mean()
        std = overall_team_stats[col].std()
        normalized_df[col] = (mean - overall_team_stats[col]) / std
    
    # Rank the teams based on normalized values
    ranked_df = normalized_df.rank(axis=0, method='min', ascending=False).astype(int)
    
    # Calculate the overall rank as the mean of individual ranks
    ranked_df['overallrank'] = ranked_df.mean(axis=1).rank(method='min', ascending=True).astype(int)
    
    # Sort the DataFrame based on the overall rank
    ranked_df = ranked_df.sort_values(by='overallrank')
    
    return normalized_df, ranked_df
normalized_df1,ranked_df1=normalization(cols_increase,cols_decrease,overall_team_stats1)
normalized_df2,ranked_df2=normalization(cols_increase,cols_decrease,overall_team_stats2)
normalized_df1.head()

In [ ]:
ranked_df1.head()

In [ ]:
ranked_df2.head()

In [ ]:
normalized_df1=normalized_df1.reindex(ranked_df1.index)

plt.figure(figsize=(12, 8))
sns.heatmap(normalized_df1, annot=ranked_df1.drop(columns='overallrank'), fmt='d', cmap='RdYlGn', linewidths=.5, center=0)

# Title and labels
plt.title('Overall Team Stats Heatmap (Ranked)')
plt.xlabel('Statistics')
plt.ylabel('Country')

# Show the plot
plt.show()

In [ ]:
df_bat_pp=get_battingstats((df[df['over']<=5]))
df_bat_mid=get_battingstats(df[(df['over']>=6)&(df['over']<=14)])
df_bat_death=get_battingstats(df[(df['over']>=15)])
df_bowl_pp=get_bowling_stats(df[df['over']<=5])
df_bowl_mid=get_bowling_stats(df[(df['over']>=6)&(df['over']<=14)])
df_bowl_death=get_bowling_stats(df[(df['over']>=15)])

In [ ]:
df.columns

In [ ]:
#df=overall_team_stats
# Normalize the data
normalized_df = (overall_team_stats1 - overall_team_stats1.min()) / (overall_team_stats1.max() - overall_team_stats1.min())

# Calculate the average values for normalized data
averages = normalized_df.mean()

# Plot
attributes = overall_team_stats1.columns
num_vars = len(attributes)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # Complete the loop

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

# Draw one axe per variable and add labels
plt.xticks(angles[:-1], attributes)

# Plot each individual country
for country in normalized_df.index:
    values = normalized_df.loc[country].tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=country)
    ax.fill(angles, values, alpha=0.1)

# Plot the average values
average_values = averages.tolist()
average_values += average_values[:1]
ax.plot(angles, average_values, linewidth=2, linestyle='dotted', label='Average', color='blue')

# Fill area for the average values
ax.fill(angles, average_values, 'b', alpha=0.1)

# Add a legend
plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

# Show the plot
plt.show()

In [ ]:
def spider_plot_stats(normalized_df,country):

    # Calculate the average values for normalized data
    averages = normalized_df.mean()

    # Plot
    attributes = normalized_df.columns
    num_vars = len(attributes)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]  # Complete the loop

    fig, ax = plt.subplots(figsize=(7,7), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable and add labels
    plt.xticks(angles[:-1], attributes)

    # Plot each individual country
    #for country in normalized_df.index:
    values = normalized_df.loc[country].tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=country)
    ax.fill(angles, values, alpha=0.1)

    # Plot the average values
    average_values = averages.tolist()
    average_values += average_values[:1]
    ax.plot(angles, average_values, linewidth=2, linestyle='dotted', label='Average', color='blue')

    # Fill area for the average values
    ax.fill(angles, average_values, 'b', alpha=0.1)

    # Add a legend
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

    # Show the plot
    plt.show()


In [ ]:
normalized_df,_=normalization(['bat_avg','bat_sr','boundary%'],['dot%','economy','bowl_avg','bowl_sr'],overall_team_stats1)
for country in normalized_df.index:
    spider_plot_stats(normalized_df,country)

In [ ]:
df_bat_pp['Phase'] = 'pp'
df_bat_mid['Phase'] = 'mid'
df_bat_death['Phase'] = 'death'
df_bowl_pp['Phase'] = 'pp'
df_bowl_mid['Phase'] = 'mid'
df_bowl_death['Phase'] = 'death'

In [ ]:
# Function to normalize and calculate the 'bowl' column
def normalize_and_calculate_bowl(df):
    df_norm, _ = normalization([], ['economy', 'bowl_avg'], df[['economy', 'bowl_avg']])
    for col in df_norm.columns:
        df.loc[:, col] = df_norm[col]
    df['bowl'] = round(0.5 * df['economy'] + 0.5 * df['bowl_avg'], 2)
    df.drop(columns=['economy', 'bowl_avg'], inplace=True)
    return df

# Function to normalize and calculate the 'bat' column
def normalize_and_calculate_bat(df):
    df_norm, _ = normalization(['bat_avg', 'bat_sr'], [], df[['bat_avg', 'bat_sr']])
    for col in df_norm.columns:
        df.loc[:, col] = df_norm[col]
    df['bat'] = round(0.5 * df['bat_avg'] + 0.5 * df['bat_sr'], 2)
    df.drop(columns=['bat_avg', 'bat_sr'], inplace=True)
    return df

# Function to separate, normalize, and calculate phases for given dataframes
def process_phases(df, team_stats1, team_stats2, normalize_func):
    df1 = df[df['Country'].isin(team_stats1.index)].copy()
    df2 = df[df['Country'].isin(team_stats2.index)].copy()

    df1 = normalize_func(df1)
    df2 = normalize_func(df2)
    
    return df1, df2

# Concatenate the bowling data and drop unnecessary columns
df_phases_bowl = pd.concat([df_bowl_pp, df_bowl_mid, df_bowl_death], ignore_index=True)
df_phases_bowl.drop(columns=['matches_bowled', 'bowl_sr'], inplace=True)

# Process bowling phases
df_phases_bowl1, df_phases_bowl2 = process_phases(df_phases_bowl, overall_team_stats1, overall_team_stats2, normalize_and_calculate_bowl)

# Concatenate the batting data and drop unnecessary columns
df_phases_bat = pd.concat([df_bat_pp, df_bat_mid, df_bat_death], ignore_index=True)
df_phases_bat.drop(columns=['matches_batted', 'dot%', 'boundary%'], inplace=True)

# Process batting phases
df_phases_bat1, df_phases_bat2 = process_phases(df_phases_bat, overall_team_stats1, overall_team_stats2, normalize_and_calculate_bat)

# Display the first few rows of the processed dataframes
print(df_phases_bowl2.head())
print(df_phases_bat1.head())

In [ ]:
# Initialize final DataFrames
df_phases1 = pd.DataFrame(columns=['Country', 'batting_pp', 'bowling_pp', 'batting_mid', 'bowling_mid', 'batting_death', 'bowling_death'])
df_phases2 = pd.DataFrame(columns=['Country', 'batting_pp', 'bowling_pp', 'batting_mid', 'bowling_mid', 'batting_death', 'bowling_death'])

# Assign unique countries
df_phases1['Country'] = df_phases_bowl1['Country'].unique()
df_phases2['Country'] = df_phases_bowl2['Country'].unique()

# Function to map phase data
def map_phase_data(df_phases, df_bat, df_bowl, phase):
    bat_phase = df_bat[df_bat['Phase'] == phase].set_index('Country')['bat']
    bowl_phase = df_bowl[df_bowl['Phase'] == phase].set_index('Country')['bowl']
    df_phases[f'batting_{phase}'] = df_phases['Country'].map(bat_phase)
    df_phases[f'bowling_{phase}'] = df_phases['Country'].map(bowl_phase)

# Map phases for both DataFrames
for phase in ['pp', 'mid', 'death']:
    map_phase_data(df_phases1, df_phases_bat1, df_phases_bowl1, phase)
    map_phase_data(df_phases2, df_phases_bat2, df_phases_bowl2, phase)

# Calculate overall score
df_phases1['overall'] = round(df_phases1.drop(columns=['Country']).mean(axis=1), 2)
df_phases2['overall'] = round(df_phases2.drop(columns=['Country']).mean(axis=1), 2)

# Reorder columns
desired_order = ['Country', 'overall', 'batting_pp', 'batting_mid', 'batting_death', 'bowling_pp', 'bowling_mid', 'bowling_death']
df_phases1 = df_phases1[desired_order]
df_phases2 = df_phases2[desired_order]

# Set 'Country' as the index and reorder according to ranked_df
df_phases1.set_index('Country', inplace=True)
df_phases1 = df_phases1.reindex(ranked_df.index)

df_phases2.set_index('Country', inplace=True)

# Display the first few rows of df_phases2
df_phases2.head()

In [ ]:
df_phases2.to_csv("docs/WC Analysis results/phases_bottom10.csv")
df_phases1.to_csv("docs/WC Analysis results/phases_top10.csv")

In [ ]:
for country in df_phases1.index:
    spider_plot_stats(df_phases1,country)

In [ ]:
averages = df_phases1.mean()

# Plot
attributes = df_phases1.columns
num_vars = len(attributes)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # Complete the loop

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

# Draw one axe per variable and add labels
plt.xticks(angles[:-1], attributes)

# Plot each individual country
for country in df_phases1.index:
    values = df_phases1.loc[country].tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=country)
    ax.fill(angles, values, alpha=0.1)

# Plot the average values
average_values = averages.tolist()
average_values += average_values[:1]
ax.plot(angles, average_values, linewidth=2, linestyle='dotted', label='Average', color='blue')

# Fill area for the average values
ax.fill(angles, average_values, 'b', alpha=0.1)

# Add a legend
plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

# Show the plot
plt.show()